In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.linalg import lstsq
import scipy.linalg as linalg 
from scipy.linalg import solve
from scipy.stats import norm
import math

# EFD Call,Put and Eurpean, American Stock

In [114]:
def Explicit_S_Put(S0,dt,dS,T,r,sigma,K,k): 
    M = np.arange(T / dt).astype(int)
    S_max = S0 + k * dS
    S_min = S0 
    s = np.arange(170,191)
    N = int((S_max - S_min) / dS)
    
    if N % 2 == 0:
        N = np.arange(N+1)
    else:
        N = np.arange(N)
        
    S = np.linspace(S_max,S_min,len(N))
    result = pd.DataFrame(index = S)

    p = pd.DataFrame(np.NaN,index = N, columns = np.arange(3))

    for i in range(1,len(N)-1):
        j = len(N)-i-1
        p_u = (dt * ((r*j)/2 + (sigma ** 2 * j**2)/2)) 
        p_m = (1 - dt * (sigma ** 2 * j **2 +r) ) 
        p_d = (dt * (-(r*j)/2 + (sigma ** 2 * j**2)/2))
        p.iloc[i,:] = [p_u,p_m,p_d]

    A = pd.DataFrame(np.NaN,index = N, columns = N)
    A.iloc[:,0:3] = p
    for i in range(1,len(N)-1):
        A.iloc[i] = A.iloc[i].shift(i-1)
    A.iloc[0,:] = A.iloc[1,:]
    A.iloc[-1,:] = A.iloc[-2,:]
    A = np.array(A.fillna(0))   

    F_1 = np.maximum(K-S,0)
    payoff = np.maximum(K-S,0)
    # note P6 L6
    F_1[0] = F_1[1]
    F_1[-1] = F_1[-2] + S[-2] - S[-1]
    
    for t in M:
        F = A @ F_1
        F_1[0] = F_1[1]
        # for B
        F[-1] = F[-2] + S[-2] - S[-1]
        # American option; comment out if is European
        F_1 = np.maximum(F,payoff)   

    result['price'] = F_1    
    return result[result.index.isin(s)]

Explicit_S_Put(S0=0,dt=0.002,dS=1,T=0.5,r=0.055,sigma=0.25,K=180,k=250)['price']

190.0     0.000000e+00
189.0    3.305484e+178
188.0     0.000000e+00
187.0    4.039158e+177
186.0     0.000000e+00
185.0    4.716539e+176
184.0     0.000000e+00
183.0    5.266908e+175
182.0     0.000000e+00
181.0    5.628507e+174
180.0     0.000000e+00
179.0    5.759951e+173
178.0     2.000000e+00
177.0    5.647906e+172
176.0     4.000000e+00
175.0    5.309097e+171
174.0     6.000000e+00
173.0    4.786319e+170
172.0     8.000000e+00
171.0    4.139724e+169
170.0     1.000000e+01
Name: price, dtype: float64

In [15]:
def Explicit_S_Call(S0,dt,dS,T,r,sigma,K,k): 
    M = np.arange(T / dt).astype(int)
    S_max = S0 + k * dS
    S_min = S0 
    s = np.arange(170,191)
    N = int((S_max - S_min) / dS)
    
    if N % 2 == 0:
        N = np.arange(N+1)
    else:
        N = np.arange(N)
        
    S = np.linspace(S_max,S_min,len(N))
    result = pd.DataFrame(index = S)

    p = pd.DataFrame(np.NaN,index = N, columns = np.arange(3))

    for i in range(1,len(N)-1):
        j = len(N)-i-1
        p_u = (dt * ((r*j)/2 + (sigma ** 2 * j**2)/2)) 
        p_m = (1 - dt * (sigma ** 2 * j **2 +r) ) 
        p_d = (dt * (-(r*j)/2 + (sigma ** 2 * j**2)/2))
        p.iloc[i,:] = [p_u,p_m,p_d]

    A = pd.DataFrame(np.NaN,index = N, columns = N)
    A.iloc[:,0:3] = p
    for i in range(1,len(N)-1):
        A.iloc[i] = A.iloc[i].shift(i-1)
    A.iloc[0,:] = A.iloc[1,:]
    A.iloc[-1,:] = A.iloc[-2,:]
    A = np.array(A.fillna(0))   

    F_1 = np.maximum(S-K,0)
    payoff = np.maximum(S-K,0)

    F_1[-1] = F_1[-2]
    F_1[0] = F_1[1] + S[0] - S[1]
    
    for t in M:
        F = A @ F_1
        F[-1] = F[-2]
        F[0] = F[1] + S[0] - S[1]
        # American option; comment out if is European
        F_1 = np.maximum(F,payoff)   
    result['price'] = F_1    
    return result[result.index.isin(s)]
Explicit_S_Call(S0=0,dt=0.002,dS=1,T=0.5,r=0.055,sigma=0.25,K=180,k=250)['price']


190.0     1.000000e+01
189.0    1.158649e+188
188.0     8.000000e+00
187.0    1.330791e+187
186.0     6.000000e+00
185.0    1.449153e+186
184.0     4.000000e+00
183.0    1.496173e+185
182.0     2.000000e+00
181.0    1.464580e+184
180.0     0.000000e+00
179.0    1.359231e+183
178.0     0.000000e+00
177.0    1.195890e+182
176.0     0.000000e+00
175.0    9.973851e+180
174.0     0.000000e+00
173.0    7.884052e+179
172.0     0.000000e+00
171.0    5.905807e+178
170.0     0.000000e+00
Name: price, dtype: float64

# IFD Call,Put and Eurpean, American Stock

In [91]:
def Implicit_S_Put(S0,dt,dS,T,r,sigma,K,k):
    M = np.arange(T / dt).astype(int)
    S_max = S0 + k * dS
    S_min = S0 
    s = np.arange(170,191)
    N = int((S_max - S_min) / dS)
    
    if N % 2 == 0:
        N = np.arange(N+1)
    else:
        N = np.arange(N)
        
    S = np.linspace(S_max,S_min,len(N))
    result = pd.DataFrame(index = S[1:-1])
    
    p = pd.DataFrame(np.NaN,index = N, columns = np.arange(3))  
    for i in range(1,len(N)-1):
        j = len(N)-i-1
        p_u = ((r*j)/2 + (sigma ** 2 * j**2)/2)
        p_m = - 1/dt - (sigma ** 2 * j **2 +r)
        p_d = (-(r*j)/2 + (sigma ** 2 * j**2)/2)
        p.iloc[i,:] = [p_u,p_m,p_d]
    p = p * -dt

    A = pd.DataFrame(np.NaN,index = N[1:-1], columns = N[1:-1])
    A.iloc[1:,0:3] = p[2:-1]
    for i in range(2,len(N)-2):
        A.iloc[i] = A.iloc[i].shift(i-1)
    A.iloc[0,0:2] = p.iloc[1,1:]
    A = np.array(A.fillna(0))

    B = np.maximum(K - S,0)[1:-1]
    payoff = np.maximum(K - S,0)[1:-1]
    B[-1] = S[-1] - S[-2]
    B[0] = 0

    for t in M:
        F = np.linalg.inv(A) @ B
        F[-1] = S[-1] - S[-2]
        F[0] = 0
        B = np.maximum(F,payoff)

    result['price'] = B
    return result[result.index.isin(s)]
Implicit_S_Put(S0=0,dt=0.002,dS=1,T=0.5,r=0.055,sigma=0.25,K=180,k=250)['price']

190.0     7.006547
189.0     7.318578
188.0     7.641896
187.0     7.976762
186.0     8.323438
185.0     8.682181
184.0     9.053245
183.0     9.436880
182.0     9.833331
181.0    10.242839
180.0    10.665638
179.0    11.101957
178.0    11.552019
177.0    12.016038
176.0    12.494221
175.0    12.986769
174.0    13.493871
173.0    14.015710
172.0    14.552457
171.0    15.104276
170.0    15.671317
Name: price, dtype: float64

In [116]:
def Implicit_S_Call(S0,dt,dS,T,r,sigma,K,k):
    M = np.arange(T / dt).astype(int)
    S_max = S0 + k * dS
    S_min = S0 
    s = np.arange(170,191)
    N = int((S_max - S_min) / dS)
    
    if N % 2 == 0:
        N = np.arange(N+1)
    else:
        N = np.arange(N)
        
    
    S = np.linspace(S_max,S_min,len(N))
    result = pd.DataFrame(index = S[1:-1])

    p = pd.DataFrame(np.NaN,index = N, columns = np.arange(3))  
    for i in range(1,len(N)-1):
        j = len(N)-i-1
        p_u = ((r*j)/2 + (sigma ** 2 * j**2)/2)
        p_m = - 1/dt - (sigma ** 2 * j **2 +r)
        p_d = (-(r*j)/2 + (sigma ** 2 * j**2)/2)
        p.iloc[i,:] = [p_u,p_m,p_d]
    p = p * -dt
    
    A = pd.DataFrame(np.NaN,index = N[1:-1], columns = N[1:-1])
    A.iloc[1:,0:3] = p[2:-1]
    for i in range(2,len(N)-2):
        A.iloc[i] = A.iloc[i].shift(i-1)
    A.iloc[0,0:2] = p.iloc[1,1:]
    A = np.array(A.fillna(0))

    B = np.maximum(S - K,0)[1:-1]
    payoff = np.maximum(S - K,0)[1:-1]
    B[-1] = 0
    B[0] = S[1] - S[2]

    for t in M:
        F = np.linalg.inv(A) @ B
        F[-1] = 0
        F[0] = S[1] - S[2]
        B = np.maximum(F,payoff)

    result['price'] = B
    return result[result.index.isin(s)]
Implicit_S_Call(S0=0,dt=0.002,dS=1,T=0.5,r=0.055,sigma=0.25,K=180,k=300)['price']

190.0    21.601888
189.0    20.900747
188.0    20.209854
187.0    19.529439
186.0    18.859724
185.0    18.200930
184.0    17.553268
183.0    16.916944
182.0    16.292158
181.0    15.679099
180.0    15.077952
179.0    14.488889
178.0    13.912076
177.0    13.347665
176.0    12.795801
175.0    12.256615
174.0    11.730228
173.0    11.216748
172.0    10.716269
171.0    10.228873
170.0     9.754628
Name: price, dtype: float64

# Crank Call,Put and Eurpean, American Stock

In [109]:
def Crank_S_Put(S0,dt,dS,T,r,sigma,K,k):
    M = np.arange(T / dt).astype(int)
    S_max = S0 + k * dS
    S_min = S0 
    s = np.arange(170,191)
    N = int((S_max - S_min) / dS)
    
    if N % 2 == 0:
        N = np.arange(N+1)
    else:
        N = np.arange(N)
        
    S = np.linspace(S_max,S_min,len(N))
    result = pd.DataFrame(index = S)
    
    a = pd.DataFrame(np.NaN,index = N, columns = np.arange(3))  
    for i in range(1,len(N)-1):
        j = len(N)-i-1
        a_3 = ((sigma ** 2 * j **2 + r * j)/4 ) 
        a_2 = (-1/dt - (sigma ** 2 * j **2 + r)/2) 
        a_1 = ((sigma ** 2 * j **2 - r * j)/4)  
        a.iloc[i,:] = [a_3,a_2,a_1]

    A = pd.DataFrame(np.NaN,index = N, columns = N)
    A.iloc[:,0:3] = a
    for i in range(1,len(N)-1):
        A.iloc[i] = A.iloc[i].shift(i-1)
    A.iloc[0,0:2] = [1,-1]
    A.iloc[-1,len(N)-2:] = [1,-1]
    A = np.array(A.fillna(0))

    b = pd.DataFrame(np.NaN,index = N, columns = np.arange(3))  
    
    for i in range(1,len(N)-1):
        j = len(N)-i-1
        b_3 = ((sigma ** 2 * j **2 + r * j)/4) 
        b_2 = (1/dt - (sigma ** 2 * j **2 + r)/2) 
        b_1 = ((sigma ** 2 * j **2 - r * j)/4) 
        b.iloc[i,:] = [-b_3,-b_2,-b_1]

    B = pd.DataFrame(np.NaN,index = N, columns = N)
    B.iloc[:,0:3] = b
    for i in range(1,len(N)-1):
        B.iloc[i] = B.iloc[i].shift(i-1)
    B = np.array(B.fillna(0))

    payoff = np.maximum(K - S,0)
    P_1 = np.maximum(K - S,0)
    Z_1 = B @ P_1
    Z_1[0] = 0
    Z_1[-1] = S[-2] - S[-1]
    for t in M:
        Z_1 = B @ P_1
        Z_1[0] = 0
        Z_1[-1] = S[-1] - S[-2]
        F = np.linalg.inv(A) @ Z_1
        P_1 = np.maximum(F,payoff)

    result['price'] = P_1
    return result[result.index.isin(s)]
Crank_S_Put(S0=0,dt=0.002,dS=1,T=0.5,r=0.055,sigma=0.25,K=180,k=250)['price']

190.0     7.021330
189.0     7.333030
188.0     7.656058
187.0     7.990672
186.0     8.337128
185.0     8.695677
184.0     9.066569
183.0     9.450051
182.0     9.846362
181.0    10.255742
180.0    10.678420
179.0    11.114622
178.0    11.564569
177.0    12.028473
176.0    12.506538
175.0    12.998963
174.0    13.505936
173.0    14.027638
172.0    14.564240
171.0    15.115902
170.0    15.682778
Name: price, dtype: float64

In [112]:
def Crank_S_Call(S0,dt,dS,T,r,sigma,K,k):
    M = np.arange(T / dt).astype(int)
    S_max = S0 + k * dS
    S_min = S0 
    s = np.arange(170,191)
    N = int((S_max - S_min) / dS)
    
    if N % 2 == 0:
        N = np.arange(N+1)
    else:
        N = np.arange(N)
        
    S = np.linspace(S_max,S_min,len(N))
    result = pd.DataFrame(index = S)
    
    a = pd.DataFrame(np.NaN,index = N, columns = np.arange(3))  
    for i in range(1,len(N)-1):
        j = len(N)-i-1
        a_3 = ((sigma ** 2 * j **2 + r * j)/4 ) 
        a_2 = (-1/dt - (sigma ** 2 * j **2 + r)/2) 
        a_1 = ((sigma ** 2 * j **2 - r * j)/4)  
        a.iloc[i,:] = [a_3,a_2,a_1]

    A = pd.DataFrame(np.NaN,index = N, columns = N)
    A.iloc[:,0:3] = a
    for i in range(1,len(N)-1):
        A.iloc[i] = A.iloc[i].shift(i-1)
    A.iloc[0,0:2] = [1,-1]
    A.iloc[-1,len(N)-2:] = [1,-1]
    A = np.array(A.fillna(0))

    b = pd.DataFrame(np.NaN,index = N, columns = np.arange(3))  
    
    for i in range(1,len(N)-1):
        j = len(N)-i-1
        b_3 = ((sigma ** 2 * j **2 + r * j)/4) 
        b_2 = (1/dt - (sigma ** 2 * j **2 + r)/2) 
        b_1 = ((sigma ** 2 * j **2 - r * j)/4) 
        b.iloc[i,:] = [-b_3,-b_2,-b_1]

    B = pd.DataFrame(np.NaN,index = N, columns = N)
    B.iloc[:,0:3] = b
    for i in range(1,len(N)-1):
        B.iloc[i] = B.iloc[i].shift(i-1)
    B = np.array(B.fillna(0))

    payoff = np.maximum(S-K,0)
    P_1 = np.maximum(S-K,0)
    
    for t in M:
        Z_1 = B @ P_1
        Z_1[0] = 0
        Z_1[-1] = S[-1] - S[-2]
        F = np.linalg.inv(A) @ Z_1
        P_1 = np.maximum(F,payoff)

    result['price'] = P_1
    return result[result.index.isin(s)]
Crank_S_Call(S0=0,dt=0.002,dS=1,T=0.5,r=0.055,sigma=0.25,K=180,k=300)['price']
    

190.0    21.624174
189.0    20.921838
188.0    20.229826
187.0    19.548361
186.0    18.877663
185.0    18.217944
184.0    17.569411
183.0    16.932267
182.0    16.306705
181.0    15.692911
180.0    15.091066
179.0    14.501338
178.0    13.923888
177.0    13.358867
176.0    12.806416
175.0    12.266663
174.0    11.739727
173.0    11.225712
172.0    10.724712
171.0    10.236806
170.0     9.762061
Name: price, dtype: float64